<a href="https://colab.research.google.com/github/rasmusli/EDAN70-relationextraction/blob/master/BioBERTEnvironment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BioBERT Environment Setup

In [0]:
!git clone https://github.com/dmis-lab/biobert.git

Cloning into 'biobert'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 308 (delta 15), reused 8 (delta 4), pack-reused 278
Receiving objects: 100% (308/308), 502.04 KiB | 2.40 MiB/s, done.
Resolving deltas: 100% (177/177), done.


In [0]:
!pip install -r biobert/requirements.txt

All requirements downloaded. Next step is to install CUDA. Specifically version 9.0 for compability with Tensorflow.

In [0]:
!sudo dpkg -i drive/My\ Drive/project_biobert/installation_files/cuda-repo-ubuntu1704_9.0.176-1_amd64.deb
!sudo apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!sudo apt-get update
!sudo apt-get install cuda-9-0 #REMEMBER YOU HAVE TO ANSWER THE PROMPT!